In [6]:
from DSPN import settoset, hungarian_loss 
import scipy.optimize
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import math
import numpy as np 
from torch.utils.data import Dataset
import pandas as pd 
import matplotlib
from matplotlib import pyplot as plt
from fspool import FSPool
class FSEncoder(nn.Module):
    def __init__(self, input_channels, output_channels, dim,set_dim, mask = False):
        super().__init__()
        self.set_dim = set_dim
        n_out = 128
        if mask == True: 
            self.mask = 1 
        else: 
            self.mask = 0
        self.conv = nn.Sequential(
            nn.Conv1d(input_channels + self.mask, dim, 1),
           ## nn.ReLU(),
           ## nn.Conv1d(dim, dim, 1),
            nn.ReLU(),
            nn.Conv1d(dim, output_channels, 1),
        )
        self.pool = FSPool(output_channels, n_out, relaxed=False)

    def forward(self, x, mask=None):
        ##mask = mask.unsqueeze(1)
        ##x = torch.cat([x, mask], dim=1)  # include mask as part of set
        x = x.reshape((self.set_dim[1],-1)).unsqueeze(2).permute(2,0,1)

        x = self.conv(x)
        x = x / x.size(2)  # normalise so that activations aren't too high with big sets
        x, _ = self.pool(x)
        return x

    
    
import csv
import json 
class newobjectdataset(Dataset): 
    def __init__(self, data_root, label_dim = 1):
      
        self.data = []
        
        self.label_dim = label_dim
        results = []
        with open(data_root) as csvfile:
            reader = csv.reader(csvfile, delimiter = "|") # change contents to floats
            for row in reader: # each row is a list
                dataentry = json.loads(row[0])
                results.append(dataentry)
        self.data = results
        
    def __getitem__(self,idx): 
        return torch.Tensor(self.data[2*idx]), torch.Tensor(self.data[2*idx + 1])
    def __len__(self): 
        return int(len(self.data)/2)

In [11]:
rate = .0005

dataset = newobjectdataset("newstate.csv")
##testdata = SquareDataset("squaredatatest.csv")
setdim2 = (1,2)
setdim1 = (1,14)
iterator = torch.utils.data.DataLoader(dataset)
##testiterator = torch.utils.data.DataLoader(testdata, shuffle = True)
encoder1 = FSEncoder(14,32,32, setdim1)
encoder2 = FSEncoder(2,32,32, setdim2)
num_actions = 6
##encoder = squareencoder()
setnet = settoset(encoder1,encoder2,32,setdim2,10, masks = True)
lambda1 = 6
optimizer = optim.Adam(setnet.parameters(),lr = rate)
running_loss = 0
epochs = 12
setnet.train()
loss_func = hungarian_loss
loss_func2 = nn.MSELoss()
for e in range(epochs): 
    print(running_loss)
    running_loss = 0
    accuracy = 0
    i = 0
    torch.set_grad_enabled(True)
    for set_,targetset_ in iterator: 
        i += 1
        
        if sum(targetset_[0]) == 0: 
            continue 
        optimizer.zero_grad()
        set_.requires_grad = True
        set_ = set_[0]
        action = set_[-1*num_actions:]
        set_ = set_[0:8]
        targetset_ = targetset_[0][0:2].reshape(1,2).transpose(1,0)/10
        
        set_ = set_.reshape(int(len(set_)/8),8)
        
        set_ = torch.cat((set_,action.unsqueeze(0)),1)/10
        
        ##print(set_)
        ##print("targetset: " + str(targetset_))
        
        
        
     

        out = setnet(set_)
        ##out = setnet(set_)
        
        loss =     loss_func(out,targetset_,setdim2) ##loss_func2(angle,setnet.encoder(set_))
        
     
        loss.backward(retain_graph = True)
        optimizer.step()
        running_loss += loss
        if i == 1000: 
            ##for parameter in setnet.parameters(): print(parameter)
            print(running_loss) 
            print(loss)
            running_loss = 0
            print(out)
            i = 0 

0
tensor(69.5896, grad_fn=<AddBackward0>)
tensor(0.0464, grad_fn=<AddBackward0>)
tensor([0.3238, 1.0239], grad_fn=<SubBackward0>)
tensor(17.2722, grad_fn=<AddBackward0>)
tensor(0.0086, grad_fn=<AddBackward0>)
tensor([0.4906, 0.9206], grad_fn=<SubBackward0>)
tensor(3.8833, grad_fn=<AddBackward0>)
tensor(0.0005, grad_fn=<AddBackward0>)
tensor([0.8784, 0.9033], grad_fn=<SubBackward0>)


KeyboardInterrupt: 

tensor([[0.4000],
        [0.9000]])

tensor([0.3282, 0.8822], grad_fn=<SubBackward0>)

tensor(5.)